In [1]:
#imports for analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import re
from pathlib import Path
pd.options.display.max_rows = 10

In [4]:
a_file = open("all_results.txt")
lines_to_read = [16, 5]

for position, line in enumerate(a_file):
    if position in lines_to_read:
        print(line)

datetime:20210405-225734





In [5]:
results = pd.read_table('all_results.txt',header=0, escapechar='\\',error_bad_lines = False)

In [ ]:
with open("all_results.txt","r") as dataFl:
    lines = [line for line in dataFl]

attribute = [] #these are the keys to the dict
data = [] #this is the data that we want to map to the keys

for line in lines:
    if line.startswith("=") or ""

In [ ]:
with open("new_results.txt","r") as dataFl:
    lines = [line for line in dataFl]

attribute = []
data = []
for line in lines:
    if line.startswith("%") or 'data' in line or line=='\n': # this is a comment or the data line
        pass
    elif line.startswith("@"):
        if "relation" in line:
            relationName = line.split(" ")[1]
        elif "attribute" in line:
            attribute.append(line.split(" ")[1])
    else:
        dataLine = list(map(lambda x : x.strip(),line.split(",")))
        dataDict = {attribute[i] : dataLine[i] for i in range(len(attribute))} # each line of data is now a dictionary.
        data.append(dataDict)

print("Relation Name is : %s" %relationName)
print("Attributes are " + ','.join(attribute))
print(data)

In [ ]:
rx_dict = {
    'datetime': re.compile(r'datetime = (?P<datetime>.*)\n'),
    'hyperparameters': re.compile(r'hyperparameters = (?P<hyperparameters>\d+)\n'),
    'results': re.compile(r'results = (?P<results>\d+)\n'),  
    'wall_time': re.compile(r'wall_time = (?P<wall_time>\d+)\n'),
    
}

In [ ]:
def _parse_line(line):
    """
    Do a regex search against all defined regexes and
    return the key and match result of the first matching regex

    """

    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            return key, match
    # if there are no matches
    return None, None

In [ ]:
def parse_file(results):
    """
    Parse text at given filepath

    Parameters
    ----------
    filepath : str
        Filepath for file_object to be parsed

    Returns
    -------
    data : pd.DataFrame
        Parsed data

    """

    data = []  # create an empty list to collect the data
    # open the file and read through it line by line
    with open(results, 'r') as file_object:
        line = file_object.readline()
        while line:
            # at each line check for a match with a regex
            key, match = _parse_line(line)

            # extract school name
            if key == 'datetime':
                school = match.group('datetime')

            # extract grade
            if key == '':
                grade = match.group('grade')
                grade = int(grade)

            # identify a table header 
            if key == 'name_score':
                # extract type of table, i.e., Name or Score
                value_type = match.group('name_score')
                line = file_object.readline()
                # read each line of the table until a blank line
                while line.strip():
                    # extract number and value
                    number, value = line.strip().split(',')
                    value = value.strip()
                    # create a dictionary containing this row of data
                    row = {
                        'School': school,
                        'Grade': grade,
                        'Student number': number,
                        value_type: value
                    }
                    # append the dictionary to the data list
                    data.append(row)
                    line = file_object.readline()

            line = file_object.readline()

        # create a pandas DataFrame from the list of dicts
        data = pd.DataFrame(data)
        # set the School, Grade, and Student number as the index
        data.set_index(['School', 'Grade', 'Student number'], inplace=True)
        # consolidate df to remove nans
        data = data.groupby(level=data.index.names).first()
        # upgrade Score from float to integer
        data = data.apply(pd.to_numeric, errors='ignore')
    return data

In [ ]:
result = pd.read_table('all_results.txt',header=0, escapechar='\\',error_bad_lines = False)

In [ ]:
result.head(10)